In [49]:
import os
import pandas as pd
import datetime
import warnings

warnings.filterwarnings(action='ignore')

### 함수 설명

- prepro_1
    1. 원천 데이터에 컬럼명이 없는 이슈로 인해 API에서 컬럼명을 추출하여 추가
    2. 결측치 제거
    3. 필요한 변수만 추출
    4. 변수 데이터 형변환
<br>
<br>
- get_etcPurpsCd
    1. 공용면적 중에서 공급면적에 포함해야되는 시설들을 선별
    2. 오피스텔과 오피스텔이 아닌 주거형태 데이터를 분리 (오피스텔은 모든 면적을 포함한 계약면적이 필요하지만 이외의 건축물들은 전용면적과 주거공용면적만 공급면적이 필요하기 때문)
<br>
<br>
- get_exclusive_Area
    1. 전용면적 변수 추가
<br>
<br>
- get_supply_Area
    1. 오피스텔과 이외의 건축물별 공용면적(계약면적) 변수를 추가

In [50]:
def prepro_1(df) :

    # 데이터 컬럼명 만들기
    df = df.T.reset_index(drop = False).T

    df.columns = ['area', 'bjdongCd', 'bldNm', 'block', 'bun', 'crtnDay', 'dongNm', 'etcPurps', 'etcStrct', 'exposPubuseGbCd', 'exposPubuseGbCdNm', 'flrGbCd', 'flrGbCdNm', \
                'flrNo', 'flrNoNm', 'hoNm', 'ji', 'lot', 'mainAtchGbCd', 'mainAtchGbCdNm', 'mainPurpsCd', 'mainPurpsCdNm', 'mgmBldrgstPk', 'naBjdongCd', 'naMainBun', 'naRoadCd', \
                'naSubBun', 'naUgrndCd', 'newPlatPlc', 'platGbCd', 'platPlc', 'regstrGbCd', 'regstrGbCdNm', 'regstrKindCd', 'regstrKindCdNm', 'rnum', 'sigunguCd', 'splotNm', 'strctCd', 'strctCdNm']

    df = df.reset_index(drop = True)
    
    # 결측치 제거
    df = df.dropna(axis = 0)

    # 불필요한 변수 제거
    df = df[['sigunguCd', 'bjdongCd', 'mgmBldrgstPk', 'platPlc', 'newPlatPlc', 'bldNm', 'mainAtchGbCd', 'exposPubuseGbCd', 'exposPubuseGbCdNm','mainAtchGbCdNm', 'mainPurpsCd', 'mainPurpsCdNm', 'area', 'etcPurps']]
    
    # 변수에 맞게 데이터 형변환 필요
    int_type_list = ['sigunguCd', 'bjdongCd', 'mainAtchGbCd', 'exposPubuseGbCd', 'area']

    for col in int_type_list :
        df[col] = df[col].apply(pd.to_numeric, errors = 'coerce').dropna().astype(float)

    return df

In [51]:
def get_etcPurpsCd (df) :

    # 주거공용면적 리스트 생성
    common_area_list = ['계단', '계단실', '로비', '벽체', '내벽', '외벽', '복도', '승강기', '엘리베이터', 'EL', 'El', '홀', '지하실', '지하층', '코아', '코어', 'Core', '공용', '공유', '공용면적', '공유면적', \
                        '주거공용', '옥탑', '홀', '계당', 'E.V', '게단실', '벽채면적']
    
    # 공유면적이 맞으면 1, 맞지 않다면 0으로 표기할 컬럼 생성 (초기값은 0으로 컬럼 생성)
    df['etcPurpsCd'] = 0

    # for문으로 common_area_list에 포함된 키워드가 df의 "etcPurps"컬럼에 존재한다면 1로 변경
    for index in range(len(df)) :
        for common_name in common_area_list :
            if common_name in df.iloc[index, 13] :
                df.loc[index, 'etcPurpsCd'] = 1

    # 오피스텔과 오피스텔이 아닌 데이터 분리
    officetel_df = df[df['mainPurpsCdNm'] == '오피스텔']
    apartment_df = df[df['mainPurpsCdNm'] != '오피스텔']
    return officetel_df, apartment_df

In [52]:
def get_exclusive_Area (df) :

    # df의 전용면적을 계산하기 위해 임시 데이터프레임을 만들어 계산한다.
    temp_df = df[df['exposPubuseGbCd'] == 1]

    # 전용공간은 Pk별로 1개가 존재하는 것이 통상적이고, 데이터도 1대1로 대응되나 예외처리를 위해 Pk로 묶어서 전용면적을 산출한다.
    grouped_temp_df = temp_df.groupby(['mgmBldrgstPk'])[['area']].sum()
    grouped_temp_df.columns = ['exclusiveArea']

    df = df.drop_duplicates(['mgmBldrgstPk'])
    df = pd.merge(df, grouped_temp_df, how = 'left', on = 'mgmBldrgstPk')

    return df

In [53]:
def get_supply_Area (officetel_df, offi_df, apartment_df, apt_df) :

    # officetel 공용면적 계산
    grouped_temp_df = officetel_df.groupby('mgmBldrgstPk')[['area']].sum()
    grouped_temp_df.columns = ['supplyArea']

    offi_df = pd.merge(offi_df, grouped_temp_df, how = 'left', on = 'mgmBldrgstPk')

    # apartment 공용면적 계산

    # 아파트의 경우, 매물별로 구분하기 위한 Pk뿐만 아니라 공급면적 구분 및 주거공용 구분하기 위한 변수를 함께 그룹화한다.
    grouped_temp_df = apartment_df.groupby(['mgmBldrgstPk', 'exposPubuseGbCd', 'etcPurpsCd']).sum()
    grouped_temp_df = grouped_temp_df.reset_index()

    # exposPubuseGbCd가 2(공급면적)이고 etcPurpsCd가 0인(주거공용면적이 아닌) row 데이터 제거
    remove_list = grouped_temp_df[(grouped_temp_df['exposPubuseGbCd'] == 2) & (grouped_temp_df['etcPurpsCd'] == 0)].index
    grouped_temp_df = grouped_temp_df.drop(remove_list, axis = 0)
    grouped_temp_df = grouped_temp_df.reset_index(drop = True)

    # 공용면적에 포함하지 않는 데이터를 제거하였으므로 Pk로 그룹화하여 area 합계를 구하면 아파트의 공급면적이 된다.
    temp_grouped_apt_df = grouped_temp_df.groupby('mgmBldrgstPk')[['area']].sum()
    temp_grouped_apt_df.columns = ['supplyArea']

    apt_df = pd.merge(apt_df, temp_grouped_apt_df, how = 'left', on = 'mgmBldrgstPk')
    
    area_df = pd.concat([apt_df, offi_df], axis = 0)
    return area_df

In [54]:
def prepro_2 (area_df) :
    
    # 공용면적만 존재하는 건축물 제거
    remove_list = area_df[area_df['exposPubuseGbCd'] == 2].index
    area_df = area_df.drop(remove_list, axis = 0)
    area_df = area_df.reset_index(drop = True)

    # 불필요한 변수 제거
    area_df = area_df[['sigunguCd', 'bjdongCd', 'mgmBldrgstPk', 'platPlc', 'newPlatPlc', 'bldNm', 'mainPurpsCdNm', 'etcPurps', 'exclusiveArea', 'supplyArea']]

    mainPurps_list = ['공동주택', '다세대주택', '다가구주택', '단독주택', '아파트', '연립주택', '오피스텔']

    # for문으로 mainPurps_list 포함된 키워드가 df의 "mainPurpsCdNm"컬럼에 존재한다면 해당 index 저장
    is_index = []

    for index in range(len(area_df)) :
        for mainPurps_name in mainPurps_list :
            if mainPurps_name in [area_df.iloc[index, 6]] :
                is_index.append(index)
    
    # mainPurps_list에 포함되지 않는 index 제거
    exclude_index = list(set([i for i in range(len(area_df))]) - set(is_index))
    area_df = area_df.drop(exclude_index, axis = 0)
    return area_df

### 건축물 대장 전처리 프로세스

1. 지역별 파일 주소 추출
2. 전처리
3. 전용면적, 공급면적(계약면적) 변수 추가
4. 지역별 통합 df에 적재
5. 저장

In [55]:
file_path = 'C:/Users/LeeJH/OneDrive - 코리아리서치인터내셔널/바탕 화면/onebite_supply/data/서울특별시_1/'

In [56]:
paths_list = [
    os.path.join(file_path,f) for f in os.listdir(file_path)
]
paths_list

['C:/Users/LeeJH/OneDrive - 코리아리서치인터내셔널/바탕 화면/onebite_supply/data/서울특별시_1/서울특별시 강남구(개포-신사).csv',
 'C:/Users/LeeJH/OneDrive - 코리아리서치인터내셔널/바탕 화면/onebite_supply/data/서울특별시_1/서울특별시 강남구(압구정-청담).csv',
 'C:/Users/LeeJH/OneDrive - 코리아리서치인터내셔널/바탕 화면/onebite_supply/data/서울특별시_1/서울특별시 강동구.csv',
 'C:/Users/LeeJH/OneDrive - 코리아리서치인터내셔널/바탕 화면/onebite_supply/data/서울특별시_1/서울특별시 강북구.csv',
 'C:/Users/LeeJH/OneDrive - 코리아리서치인터내셔널/바탕 화면/onebite_supply/data/서울특별시_1/서울특별시 강서구(가양-마곡).csv',
 'C:/Users/LeeJH/OneDrive - 코리아리서치인터내셔널/바탕 화면/onebite_supply/data/서울특별시_1/서울특별시 강서구(방화-화곡).csv',
 'C:/Users/LeeJH/OneDrive - 코리아리서치인터내셔널/바탕 화면/onebite_supply/data/서울특별시_1/서울특별시 관악구.csv',
 'C:/Users/LeeJH/OneDrive - 코리아리서치인터내셔널/바탕 화면/onebite_supply/data/서울특별시_1/서울특별시 광진구.csv',
 'C:/Users/LeeJH/OneDrive - 코리아리서치인터내셔널/바탕 화면/onebite_supply/data/서울특별시_1/서울특별시 구로구.csv',
 'C:/Users/LeeJH/OneDrive - 코리아리서치인터내셔널/바탕 화면/onebite_supply/data/서울특별시_1/서울특별시 금천구.csv',
 'C:/Users/LeeJH/OneDrive - 코리아리서치인터내셔널/바탕 화면/onebite_supply/data/서울특별시

In [57]:
start_time = datetime.datetime.now()
total_df = pd.DataFrame(columns = ['sigunguCd', 'bjdongCd', 'mgmBldrgstPk', 'platPlc', 'newPlatPlc', 'bldNm', 'mainPurpsCdNm', 'etcPurps', 'exclusiveArea', 'supplyArea'])
total_data_cnt = 0

In [58]:
for i in range(len(paths_list)) :
    print(paths_list[i])
    df = pd.read_csv(paths_list[i], encoding = 'cp949', low_memory = False)

    # 변수 추가를 위한 전처리
    df = prepro_1(df)
    total_data_cnt += len(df)

    # 주거공용시설 구분
    officetel_df, apartment_df = get_etcPurpsCd(df)

    # 전용면적 변수 추가
    offi_df = get_exclusive_Area(officetel_df)
    apt_df = get_exclusive_Area(apartment_df)
    
    # 공급면적 변수 추가
    area_df = get_supply_Area (officetel_df, offi_df, apartment_df, apt_df)

    # 원하는 데이터만 추출하기 위한 전처리
    area_df = prepro_2 (area_df)

    # 전처리가 끝난 데이터들을 지역별 병합
    total_df = pd.concat([total_df, area_df], axis = 0)

# 전용 & 공급면적 round(2) 진행 
total_df['exclusiveArea'] = total_df['exclusiveArea'].apply(lambda x : round(x,2))
total_df['supplyArea'] = total_df['supplyArea'].apply(lambda x : round(x,2))

# pk를 제외한 변수들에 대해서 중복으로 존재하는 데이터 제거
total_df = total_df.drop_duplicates(['sigunguCd', 'bjdongCd', 'platPlc', 'newPlatPlc', 'bldNm', 'mainPurpsCdNm', 'etcPurps', 'exclusiveArea', 'supplyArea'])

total_df = total_df.reset_index(drop = True)
    

C:/Users/LeeJH/OneDrive - 코리아리서치인터내셔널/바탕 화면/onebite_supply/data/서울특별시_1/서울특별시 강남구(개포-신사).csv
C:/Users/LeeJH/OneDrive - 코리아리서치인터내셔널/바탕 화면/onebite_supply/data/서울특별시_1/서울특별시 강남구(압구정-청담).csv
C:/Users/LeeJH/OneDrive - 코리아리서치인터내셔널/바탕 화면/onebite_supply/data/서울특별시_1/서울특별시 강동구.csv
C:/Users/LeeJH/OneDrive - 코리아리서치인터내셔널/바탕 화면/onebite_supply/data/서울특별시_1/서울특별시 강북구.csv
C:/Users/LeeJH/OneDrive - 코리아리서치인터내셔널/바탕 화면/onebite_supply/data/서울특별시_1/서울특별시 강서구(가양-마곡).csv
C:/Users/LeeJH/OneDrive - 코리아리서치인터내셔널/바탕 화면/onebite_supply/data/서울특별시_1/서울특별시 강서구(방화-화곡).csv
C:/Users/LeeJH/OneDrive - 코리아리서치인터내셔널/바탕 화면/onebite_supply/data/서울특별시_1/서울특별시 관악구.csv
C:/Users/LeeJH/OneDrive - 코리아리서치인터내셔널/바탕 화면/onebite_supply/data/서울특별시_1/서울특별시 광진구.csv
C:/Users/LeeJH/OneDrive - 코리아리서치인터내셔널/바탕 화면/onebite_supply/data/서울특별시_1/서울특별시 구로구.csv
C:/Users/LeeJH/OneDrive - 코리아리서치인터내셔널/바탕 화면/onebite_supply/data/서울특별시_1/서울특별시 금천구.csv
C:/Users/LeeJH/OneDrive - 코리아리서치인터내셔널/바탕 화면/onebite_supply/data/서울특별시_1/서울특별시 노원구.csv
C:/Users/LeeJH/OneDrive -

In [59]:
end_time = datetime.datetime.now()

In [60]:
total_df.to_csv('서울특별시_1' + '_add_area.csv')